<p style="float:right">
<img src="nasa.png" style="display:inline" />
</p>

# Geographical Subsetting with OPeNDAP

For this demonstration, we will examine the [_MEaSUREs Northern Hemisphere Terrestrial Snow Cover Extent Daily 25km EASE-Grid 2.0_](http://nsidc.org/data/docs/measures/nsidc-0530/) dataset from the [NASA MEaSUREs project](https://earthdata.nasa.gov/community/community-data-system-programs/measures-projects). This dataset is managed by [NSIDC DAAC](http://nsidc.org/data/measures/) and made available via, among other methods, the [OPeNDAP](https://en.wikipedia.org/wiki/OPeNDAP) protocol.

### Determining grid coordinates for a geographical area

Let's say that we're interested only in data pertaining to Iceland. We can definte geographical bounds for our area of interest:

In [ ]:
bounds = {'n': 67, 's': 63, 'e': -13, 'w': -25, }
#lat_south = 63
#lat_north = 67
#lon_west = -25
#lon_east = -13

As in the notebook _Variable Subsetting with OPeNDAP_, we will use the Python `netCDF4` package's built-in OPeNDAP client to access snow-cover data from June 1, 2012. In this case, we will request only the **latitude** and **longitude** variables:

In [ ]:
import netCDF4

url0 = (
    'http://n5eil01u.ecs.nsidc.org:80/opendap/'
    'MEASURES/NSIDC-0530.001/2012.06.01/nhtsd25e2_20120601_v01r01.nc?'
    'latitude[0:1:719][0:1:719],'
    'longitude[0:1:719][0:1:719]'
)

dataset0 = netCDF4.Dataset(url0)

for v in dataset0.variables:
    print(v, dataset0.variables[v].shape)

Instead of full 720 x 720 grids, we would like to constrain future OPeNDAP requests so that they return data only for the Iceland area. To do so, we'll use NumPy to obtain lists of grid rows and columns where both `latitude` and `longitude` values fall within the defined Iceland area, then take the minima and maxima of each list to construct bounding-box values we can pass to OPeNDAP:

In [ ]:
import numpy as np

lat = dataset0.variables['latitude'][:, :]
lon = dataset0.variables['longitude'][:, :]
row, col = np.where((lat >= bounds['s']) & (lat <= bounds['n']) & (lon >= bounds['w']) & (lon <= bounds['e']))
bbox = np.min(row), np.max(row), np.min(col), np.max(col)
print(bbox)

Now we can use our bounding-box values as constraints in our OPeNDAP URL to select just the rows and columnsthat correspond to Iceland. The OPeNDAP constraints are given in `lower_bound:stride:upper_bound` form:

In [ ]:
url1 = (
    'http://n5eil01u.ecs.nsidc.org/opendap/'
    'MEASURES/NSIDC-0530.001/2012.06.01/nhtsd25e2_20120601_v01r01.nc?'
    'latitude[453:1:475][310:1:336],'
    'longitude[453:1:475][310:1:336],'
    'merged_snow_cover_extent[0:1:0][453:1:475][310:1:336]'
)

dataset1 = netCDF4.Dataset(url1)

for v in dataset1.variables:
    print(v, dataset1.variables[v].shape)

That's a lot less data!

Finally, we can plot our data on a map of Iceland, using code similar to that shown in the notebook _Variable Subsetting with OPeNDAP_:

In [ ]:
%matplotlib inline
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

def snow_ice_plot(snow_ice_map, lat, lon, condition, color, size=1):
    row, col = np.where(condition)
    x, y = snow_ice_map(lon[row, col], lat[row, col])
    snow_ice_map.scatter(x, y, s=size, color=color)

lat0 = dataset1.variables['latitude'][:, :]
lon0 = dataset1.variables['longitude'][:, :]
msce_jan = dataset1.variables['merged_snow_cover_extent'][0, :, :]

plt.figure(figsize=(10, 10))
iceland_jan = Basemap(
    projection='merc',
    resolution='i',
    urcrnrlat=bounds['n'],
    llcrnrlat=bounds['s'],
    urcrnrlon=bounds['e'],
    llcrnrlon=bounds['w'],
)
iceland_jan.drawcoastlines()

snow_ice_plot(iceland_jan, lat0, lon0, (msce_jan >= 10) & (msce_jan <= 16), 'Cyan', 500)
snow_ice_plot(iceland_jan, lat0, lon0, msce_jan == 30, 'Blue', 500)

### Using a data-access class

As a simplification, we can make use of a prebuilt Pyhton class, `NSIDC0530`, to access remote data. The class constructor takes year, date and month arguments, as well as an optional list of desired variables. (Note that the `latitude` and `longitude` variables are provided with the class and need not be requested.) A second optional argument specifies a dict of east-longitude, west-longitude, south-latitude and north-latitude bounds

Here, we request the `merged_snow_cover_extent` data for September 1, 2012, then plot it with code otherwise nearly identical to that used in the previous example:

In [ ]:
from nsidc0530.interface import NSIDC0530

sep_data = NSIDC0530(2012, 9, 1, ['merged_snow_cover_extent'], bounds)

lat1 = sep_data.latitude
lon1 = sep_data.longitude
msce_sep = sep_data.merged_snow_cover_extent

plt.figure(figsize=(10, 10))
iceland_sep = Basemap(
    projection='merc',
    resolution='i',
    urcrnrlat=bounds['n'],
    llcrnrlat=bounds['s'],
    urcrnrlon=bounds['e'],
    llcrnrlon=bounds['w'],
)
iceland_sep.drawcoastlines()

snow_ice_plot(iceland_sep, lat1, lon1, (msce_sep >= 10) & (msce_sep <= 16), 'Cyan', 500)
snow_ice_plot(iceland_sep, lat1, lon1, msce_sep == 30, 'Blue', 500)